In [1]:
import math
from skimage import io, color
from skimage.transform import resize
import numpy as np
from tqdm import tqdm

In [2]:
# function which returns an object of class SuperPixel
def make_superPixel(h, w,img):
    return SuperPixels(h, w,img[h,w][0],img[h,w][1],img[h,w][2])

In [3]:
# To define the initial cluster centers distanced at S
def initial_cluster_center(S,img,img_h,img_w,clusters):
    h = S // 2
    w = S // 2
    while h < img_h:
        while w < img_w:
            clusters.append(make_superPixel(h, w,img))
            w += S
        w = S // 2
        h += S
    return clusters

In [4]:
# function to calculate gradient at each pixel 
def calc_gradient(h, w,img,img_w,img_h):
    if w + 1 >= img_w:
        w = img_w - 2
    if h + 1 >= img_h:
        h = img_h - 2
    grad = img[w + 1, h + 1][0] - img[w, h][0] + img[w + 1, h + 1][1] - img[w, h][1] + img[w + 1, h + 1][2] - img[w, h][2]
    return grad

In [5]:
# function which reassign the cluster center to the pixel having the lowest gradient
def reassign_cluster_center_acc_to_grad(clusters,img):
    for c in clusters:
        cluster_gradient = calc_gradient(c.h, c.w,img,img_w,img_h)
        for dh in range(-1, 2):
            for dw in range(-1, 2):
                H = c.h + dh
                W = c.w + dw
                new_gradient = calc_gradient(H,W, img,img_w,img_h)
                if new_gradient < cluster_gradient:
                    c.update(H, W,img[H,W][0], img[H,W][1],img[H,W][2])
                    c_gradient = new_gradient


In [6]:
'''
function to assign pixels to the nearest cluster using the Euclidean distance involving both color and spatial
proximity.
'''
def assign_pixels_to_cluster(clusters,S,img,img_h,img_w,tag,dis):
    for c in clusters:
        for h in range(c.h - 2 * S, c.h + 2 * S):
            if h < 0 or h >= img_h: continue
            for w in range(c.w - 2 * S, c.w + 2 * S):
                if w < 0 or w >= img_w: continue
                l, a, b = img[h,w]
                Dc = math.sqrt(math.pow(l - c.l, 2) + math.pow(a - c.a, 2) + math.pow(b - c.b, 2))
                Ds = math.sqrt(math.pow(h - c.h, 2) + math.pow(w - c.w, 2))
                D = math.sqrt(math.pow(Dc / m, 2) + math.pow(Ds /S, 2))
                if D < dis[h,w]:
                    if (h, w) not in tag:
                        tag[(h, w)] = c
                        c.pixels.append((h, w))
                    else:
                        tag[(h, w)].pixels.remove((h, w))
                        tag[(h, w)] = c
                        c.pixels.append((h, w))
                    dis[h, w] = D

In [7]:
# function to replace the cluster center with the mean of the pixels contained in the cluster
def update_cluster_mean(clusters):
    for c in clusters:
        sum_h = sum_w = number = 0
        #print("c.pixels",c.pixels)
        for p in c.pixels:
            sum_h += p[0]
            sum_w += p[1]
            number += 1
            H = sum_h // number
            W = sum_w // number
            c.update(H, W,img[H, W][0], img[H, W][1], img[H, W][2])

In [8]:
# replace the color of each pixel in a cluster by the color of the cluster's center
def avg_color_cluster(img,name,clusters):
    image = np.copy(img)
    for c in clusters:
        for p in c.pixels:
            image[p[0],p[1]][0] = c.l
            image[p[0],p[1]][1] = c.a
            image[p[0],p[1]][2] = c.b
        # To change the color of cluster center to Black
        image[c.h, c.w][0] = 0
        image[c.h, c.w][1] = 0
        image[c.h, c.w][2] = 0
    lab2rgb(name, image)

In [17]:
# function for the Simple Linear Iterative Clustering
def slic(S,img,img_h,img_w,clusters,tag,dis, imgname):
    clusters = initial_cluster_center(S,img,img_h,img_w,clusters)
    reassign_cluster_center_acc_to_grad(clusters,img)
    for i in range(10): # usually the algortihm converges within 10 iterations
        assign_pixels_to_cluster(clusters,S,img,img_h,img_w,tag,dis)
        update_cluster_mean(clusters)
        if i == 9 : # to print the output after 10 iterations
            name = 'outputs/{imgname}_out_m{m}_k{k}.png'.format(loop=i, m=m, k=k, imgname=imgname)
            avg_color_cluster(img,name, clusters)
    return clusters

In [10]:
# function to convert LAB images back to RGB and save it
def lab2rgb(path, lab_arr):
    rgb_arr = color.lab2rgb(lab_arr)
    io.imsave(path, rgb_arr)

In [11]:
# A class to initialize the super pixels, of the form - [h,y,l,a,b].
class SuperPixels(object):

    def __init__(self, h, w, l=0, a=0, b=0):
        self.update(h, w, l, a, b)
        self.pixels = []

    def update(self, h, w, l, a, b):
        self.h = h
        self.w = w
        self.l = l
        self.a = a
        self.b = b

In [ ]:
base_path = r"segmentation-images-test/"
for i in tqdm(range(1,9)):
    # read the input RGB image
    image_path = base_path + str(i) + ".jpeg"
    rgb = io.imread(image_path,plugin='matplotlib')
    print(rgb.shape)

    # input images are resized to (400 x 400) for processing
    img = resize(rgb, (400,400),anti_aliasing=True)
    print(img.shape)

    # convert RGB to LAB
    img = color.rgb2lab(img)

    k = 200  # Number of Super pixels
    m = 20  # Constant for normalizing the color proximity, range of m = [1,40]

    img_h = img.shape[0] # Image Height
    img_w = img.shape[1] # Image Width

    N = img_h * img_w  # Total number of pixels in the image
    S = int(math.sqrt(N /k)) # average size of each superpixel

    clusters = []
    tag = {}
    # initialize the distance between pixels and cluster center as infinity
    dis = np.full((img_h, img_w), np.inf)
    cluster = slic(S,img,img_h,img_w,clusters,tag,dis, str(i))

In [15]:
# superpixels
for c in cluster:
    print("H {} : W {}, l {}, a {}, b {}".format(c.h,c.w,c.l,c.a,c.b))

H 19 : W 19, l 95.57344008521166, a -1.1472062890243717, b -0.9400826199776224
H 19 : W 59, l 95.27605182153654, a -0.7869318738305542, b -1.9017024862269194
H 20 : W 98, l 95.27605182153654, a -0.7869318738305542, b -1.9017024862268972
H 19 : W 138, l 95.27605182153654, a -0.7869318738305542, b -1.9017024862268972
H 19 : W 178, l 94.57842480744429, a -0.7880330592238693, b -1.9046847362147679
H 20 : W 218, l 93.83273028781892, a -0.9353811310358617, b -2.504569649301369
H 18 : W 259, l 92.89011525690428, a -2.1054405783056396, b -2.3695090835526678
H 19 : W 300, l 91.56744992884404, a -0.8495038961719992, b -4.6811048620982065
H 19 : W 340, l 89.76804557228266, a -1.4450506949253983, b -6.009952979725397
H 20 : W 379, l 88.11389886820224, a -2.6141134431107993, b -5.882031392511355
H 59 : W 19, l 94.92737536720449, a -0.7874812946570553, b -1.903190175649283
H 61 : W 60, l 95.27605182153654, a -0.7869318738305542, b -1.9017024862269194
H 60 : W 100, l 94.92737536720449, a -0.787481294